In [2]:
import pandas as pd
import numpy as np

In [ ]:
path_strain_genome = ""

In [6]:
#这是ref genome的acns，有65703个
acns_tax_rep_path = "/data/huixingqi/sim_meta/data/taxonomy_rank/name.txt"
acns_tax_rep = pd.read_csv(acns_tax_rep_path,sep="\t",header=None)
print(len(acns_tax_rep))
acns_tax_rep.columns=["acns","sp"]
acns_tax_rep["acns"] = acns_tax_rep["acns"].apply(lambda x:x.replace("GB_",""))
acns_tax_rep["acns"] = acns_tax_rep["acns"].apply(lambda x:x.replace("RS_",""))
acns_tax_rep["acns"] = acns_tax_rep["acns"].apply(lambda x:x.split("_")[0]+"_"+x.split("_")[1])
print(acns_tax_rep[:2])

65703
              acns                          sp
0  GCF_004114995.1  s__Halorubrum_amylolyticum
1  GCF_007671685.1    s__Halorubrum_salsamenti


In [7]:
#发现有些物种的old不在了
#只要新的吧
path_link1 = "/data/huixingqi/sim_meta/data/ncbi_genome/assembly_summary_genbank.txt"
acns_link1 = pd.read_csv(path_link1,skiprows=[0],sep="\t")
# print(len(acns_link1))
# print(acns_link1[:5])
acns_link1 = acns_link1.rename(columns={'# assembly_accession': 'acns'})
# print(acns_link1[:5])
acns_link_1 = acns_link1.loc[:,["acns","taxid","ftp_path"]]
print(len(acns_link_1))
# print(acns_link_1[:5])
path_link2 = "/data/huixingqi/sim_meta/data/ncbi_genome/assembly_summary_refseq.txt"
acns_link2 = pd.read_csv(path_link2,skiprows=[0],sep="\t")
acns_link2 = acns_link2.rename(columns={'# assembly_accession': 'acns'})
acns_link_2 = acns_link2.loc[:,["acns","taxid","ftp_path"]]
print(len(acns_link_2))
acns_link = pd.concat([acns_link_1,acns_link_2],axis=0)
print(len(acns_link))

acns_link_drop = acns_link.drop_duplicates().reset_index(drop=True)
print(len(acns_link),len(acns_link_drop))




1621901
300088
1921989
1921989 1921989


In [8]:
path_all_ar ="/data/huixingqi/sim_meta/data/taxonomy_rank/ar53_taxonomy_r207.tsv"
path_all_bac ="/data/huixingqi/sim_meta/data/taxonomy_rank/bac120_taxonomy_r207.tsv"
df_all_ar = pd.read_csv(path_all_ar,header=None,sep="[\t;]")
df_all_bac = pd.read_csv(path_all_bac,header=None,sep="[\t;]")
df_all_gt = pd.concat([df_all_ar,df_all_bac],axis=0)
df_all_gt = df_all_gt.reset_index(drop=True)
print(len(df_all_ar),len(df_all_bac),len(df_all_gt))

# print(df_all_ar[:2])
df_all_gt.columns=["acns","domain","phylum","class","order","family","genus","species"]
print(len(df_all_gt))

df_all_gt["species"] = df_all_gt["species"].apply(lambda x:x.replace(" ","_"))
df_all_gt["acns"] = df_all_gt["acns"].apply(lambda x:x.replace("GB_",""))
df_all_gt["acns"] = df_all_gt["acns"].apply(lambda x:x.replace("RS_",""))
df_all_gt["acns"] = df_all_gt["acns"].apply(lambda x:x.split("_")[0]+"_"+x.split("_")[1])

print(df_all_gt[:2])
print(len(df_all_gt),df_all_gt["acns"].nunique())
print(df_all_gt["species"].nunique())
unique_acns = acns_tax_rep["acns"].unique()
mask = df_all_gt["acns"].isin(unique_acns)
mask_f = ~df_all_gt["acns"].isin(unique_acns)
print("len(unique_acns)",len(unique_acns))
print("len(mask),len(df_all_gt)",len(mask),len(df_all_gt))
df_all_gt_noGT  = df_all_gt.drop(df_all_gt[mask].index)
df_all_gt_GT  = df_all_gt.drop(df_all_gt[mask_f].index)
print("len(df_all_gt_noGT)",len(df_all_gt_noGT),len(df_all_gt)-65703)
print(len(df_all_gt_GT))
df_all_gt_noGT_acns = pd.DataFrame(df_all_gt_noGT["acns"])
print(len(df_all_gt_noGT_acns))

/tmp/ipykernel_2665357/3658395269.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_all_ar = pd.read_csv(path_all_ar,header=None,sep="[\t;]")
/tmp/ipykernel_2665357/3658395269.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_all_bac = pd.read_csv(path_all_bac,header=None,sep="[\t;]")


6062 311480 317542
317542
              acns      domain             phylum               class  \
0  GCF_000979375.1  d__Archaea  p__Halobacteriota  c__Methanosarcinia   
1  GCF_000970165.1  d__Archaea  p__Halobacteriota  c__Methanosarcinia   

                  order                 family              genus  \
0  o__Methanosarcinales  f__Methanosarcinaceae  g__Methanosarcina   
1  o__Methanosarcinales  f__Methanosarcinaceae  g__Methanosarcina   

                   species  
0  s__Methanosarcina_mazei  
1  s__Methanosarcina_mazei  
317542 317542
65703
len(unique_acns) 65703
len(mask),len(df_all_gt) 317542 317542
len(df_all_gt_noGT) 251839 251839
65703
251839


In [23]:
df_path_noGT = pd.merge(df_all_gt_noGT,acns_link_drop,on=["acns"])

df_path_noGT["path"] = df_path_noGT["ftp_path"].apply(lambda x:x+"/"+x.split("/")[-1]+"_genomic.fna.gz")
print(df_path_noGT[:2])

              acns      domain             phylum               class  \
0  GCF_000979375.1  d__Archaea  p__Halobacteriota  c__Methanosarcinia   
1  GCF_000970165.1  d__Archaea  p__Halobacteriota  c__Methanosarcinia   

                  order                 family              genus  \
0  o__Methanosarcinales  f__Methanosarcinaceae  g__Methanosarcina   
1  o__Methanosarcinales  f__Methanosarcinaceae  g__Methanosarcina   

                   species    taxid  \
0  s__Methanosarcina_mazei     2209   
1  s__Methanosarcina_mazei  1434117   

                                            ftp_path  \
0  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...   
1  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...   

                                                path  
0  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...  
1  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...  


In [ ]:
cnt_fp = 0
for fp in df_path_noGT["path"]:
    if check_file_exists(fp):
        cnt_fp+=1
    else:
        
        print(fp)
        

无法连接到服务器或请求出错
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/617/325/GCF_000617325.1_Ec06-3325/GCF_000617325.1_Ec06-3325_genomic.fna.gz
无法连接到服务器或请求出错
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/900/093/985/GCF_900093985.1_scaffolds_20160017/GCF_900093985.1_scaffolds_20160017_genomic.fna.gz
无法连接到服务器或请求出错
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/009/812/175/GCF_009812175.1_ASM981217v1/GCF_009812175.1_ASM981217v1_genomic.fna.gz
无法连接到服务器或请求出错
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/002/923/635/GCF_002923635.1_ASM292363v1/GCF_002923635.1_ASM292363v1_genomic.fna.gz
无法连接到服务器或请求出错
https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/005/382/325/GCF_005382325.1_ASM538232v1/GCF_005382325.1_ASM538232v1_genomic.fna.gz


In [22]:
url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/979/375/GCF_000979375.1_gtlEnvA5udCFS/GCF_000979375.1_gtlEnvA5udCFS.fna.gz"
response = requests.head(url)
print(response.status_code)

404


In [11]:
import requests

def check_file_exists(url):
    try:
        response = requests.head(url)
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"服务器返回状态码: {response.status_code}")
            return False
    except requests.RequestException:
        print("无法连接到服务器或请求出错")
        return False
